In [9]:
import numpy as np
import pandas as pd

from pyecharts.charts import *
from pyecharts import options as opts

from sklearn.cluster import KMeans
import joblib
from sklearn import metrics
from scipy.spatial.distance import cdist 
import time

In [10]:
user_feature = pd.read_csv('用户特征.csv', index_col=0)
author_feature = pd.read_csv('作者特征.csv', index_col=0)

In [11]:
len(user_feature)
user_feature['点赞量'].sum()

np.int64(16773)

In [12]:
user_data = user_feature[(user_feature['完整观看数']>=1)&(user_feature['浏览量']>=5)]
print(len(user_data)/len(user_feature))

0.7097514856834144


In [13]:

author_data = author_feature[(author_feature['总观完量']>=1)&(author_feature['总浏览量']>=3)]
print(len(author_data)/len(author_feature))

0.2990244347629775


In [14]:
def km(data, name):
    K = range(2, 10) # K值选取范围
    X = data # 数据
    # scores = { 'SSE': [], 'sc': [], 'sse': []}
    scores = {'sc': [], 'sse': []}
    for _k in K:
        # 初始化模型并进行聚类 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        kmeans = KMeans(n_clusters=_k, init='k-means++', random_state=0)
        kmeans.fit(X)
        _y = kmeans.predict(X) # 预测结果
        # 计算模型评估指标 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        sse = sum(np.min(cdist(X,kmeans.cluster_centers_,'euclidean'),axis=1))/X.shape[0]
        sc = metrics.silhouette_score(X, _y) # 计算轮廓系数
        joblib.dump(kmeans, f'{name}{_k}聚类.model')
        # 储存评估值 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        # scores['SSE'].append(SSE)
        scores['sse'].append(sse)
        scores['sc'].append(sc)
        print(f'聚{_k}类计算完成', end='\t')
    joblib.dump(scores, f'{name}聚类指标.score')
    print('指标储存完毕')
    return scores

In [15]:
def draw(k, sse, sc):
    chart = (
        Line(init_opts=opts.InitOpts(
            theme='light',
            width='350px',
            height='350px'
        ))
        .add_xaxis(k)
        .add_yaxis('sse', sse, yaxis_index=0, label_opts=opts.LabelOpts(is_show=False))
        .add_yaxis('sc', sc, yaxis_index=1, label_opts=opts.LabelOpts(is_show=False))
        .extend_axis(yaxis=opts.AxisOpts())
        .set_global_opts(
            title_opts=opts.TitleOpts(title='聚类效果'),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=True),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        )
    )
    return chart

In [16]:
start_time = time.time()
user_score = km(user_data, '用户')
end_time = time.time()
print('用户聚类耗时：', end_time-start_time)

聚2类计算完成	聚3类计算完成	聚4类计算完成	聚5类计算完成	聚6类计算完成	聚7类计算完成	聚8类计算完成	聚9类计算完成	指标储存完毕
用户聚类耗时： 105.145751953125


In [17]:
user_score =  joblib.load(f'用户聚类指标.score')
draw([str(x) for x in range(2,10)], user_score['sse'], user_score['sc']).render_notebook()

In [18]:

user_km = joblib.load(f'用户4聚类.model')
user_centers = pd.DataFrame(user_km.cluster_centers_, columns=user_feature.columns)
user_centers['人数']=pd.Series(user_km.predict(user_data)).value_counts()
user_centers

,浏览量,点赞量,观看作者数,观看作品数,观看作品平均时长,观看配乐数,完整观看数,去过的城市数,观看作品城市数,人数
0,16.891674,0.191183,16.609704,16.891540,11.314563,16.023050,8.166387,1.170956,14.647672,29740
1,163.284336,1.268519,155.473765,163.282407,11.079584,132.366898,54.168210,1.295525,88.876929,2595
2,66.012507,0.655094,64.052183,66.011644,11.174247,58.415849,27.607763,1.273100,47.351375,9294
3,382.953771,3.038929,354.287105,382.948905,10.968649,277.233577,92.829684,1.321168,141.182482,411


In [19]:

start_time = time.time()
user_score = km(user_data, '作者')
end_time = time.time()
print('作者聚类耗时：', end_time-start_time)

聚2类计算完成	聚3类计算完成	聚4类计算完成	聚5类计算完成	聚6类计算完成	聚7类计算完成	聚8类计算完成	聚9类计算完成	指标储存完毕
作者聚类耗时： 126.28758358955383


In [20]:

author_score =  joblib.load(f'作者聚类指标.score')
draw([str(x) for x in range(2,10)], author_score['sse'], author_score['sc']).render_notebook()

In [22]:
print("训练时的特征：", author_feature.columns)
print("预测时的特征：", author_data.columns)

训练时的特征： Index(['总浏览量', '总点赞量', '总观完量', '总作品数', '作品平均时长', '使用配乐数量', '发布作品日数',
       '创作活跃度(日)', '去过的城市数'],
      dtype='object')
预测时的特征： Index(['总浏览量', '总点赞量', '总观完量', '总作品数', '作品平均时长', '使用配乐数量', '发布作品日数',
       '创作活跃度(日)', '去过的城市数'],
      dtype='object')


In [ ]:
author_km = joblib.load(f'作者4聚类.model')
author_centers = pd.DataFrame(author_km.cluster_centers_, columns=author_feature.columns)
author_centers['人数'] = pd.Series(author_km.predict(author_data)).value_counts()
author_centers